# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks, code quality and readability (no single-letter variables, PEP8 compliant) etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark

findspark.init()

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, LongType, ShortType

sc = SparkContext(appName="HDFS Homework")
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-10 20:31:29,626 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


### Spark DF Solution

In [2]:
# reading csv into dataframe

df = spark.read.csv('hdfs:/data/clickstream.csv', header=True, sep='\t', inferSchema=True)
df.show(5) # check table structure

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



In [3]:
"""
for each user_id and session_id calculate the timestamp
of first error if any and store it into dataframe
"""

error_ts_df = (df.filter(df.event_type.contains('error'))
                 .groupBy('user_id', 'session_id')
                 .agg({'timestamp': 'min'})
                 .withColumnRenamed('min(timestamp)', 'error_ts'))

In [4]:
"""
join the error timestamp with the main dataframe
"""

df_join = df.join(error_ts_df, on=['user_id', 'session_id'], how='left')

In [5]:
"""
calculate the route with the count
"""

# filter the redundat rows based on error timestamp and type of event type
df_output = (df_join.filter((F.col('event_type') == 'page') & ((F.col('error_ts').isNull()) | (F.col('timestamp') < F.col('error_ts'))))
                    .orderBy('user_id', 'session_id', 'timestamp') # order by timestamp before creating route
                    .groupBy('user_id', 'session_id') # create route of unique user_id + session_id
                    .agg(F.concat_ws('-', F.collect_list(F.col('event_page')))) # create route by concatenating string
                    .withColumnRenamed('concat_ws(-, collect_list(event_page))', 'route') # rename colum to route
                    .groupBy('route') # group by route
                    .count() # count of routes
                    .sort(F.col('count').desc()) # sort in decending order
             )

df_output.show(10)

+-------------------+-----+
|              route|count|
+-------------------+-----+
|               main| 8090|
|       main-archive| 1095|
|        main-rabota| 1039|
|      main-internet|  880|
|         main-bonus|  865|
|          main-news|  760|
|       main-tariffs|  669|
|        main-online|  584|
|         main-vklad|  514|
|main-archive-rabota|  167|
+-------------------+-----+
only showing top 10 rows



In [7]:
# write results
(df_output.limit(30)
          .select(F.concat_ws(',', *df_output.columns))
          .write.option('header', False)
          .text("df_result", lineSep="\t"))

### Spark SQL Solution

In [8]:
# read csv and register a temprory table

df = spark.read.csv('hdfs:/data/clickstream.csv', header=True, sep='\t', inferSchema=True)
df.registerTempTable('clickstream')
df.limit(3).show()

+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|   4645|       935|      page|   archive|1696270588|
|   2251|       378|      page|    rabota|1696270591|
|   2222|       704|      page|    rabota|1696270600|
+-------+----------+----------+----------+----------+



In [9]:
'''
this query does exactly the same thing as we did in dataframe solution
innermost select query calculates the error timestamp
2nd select query concatenates page to create route for each user + session_id
3rd query just count the routes
'''

sql_out = spark.sql(
            '''
            SELECT route, COUNT(*) AS count
            FROM (SELECT concat_ws('-', sort_array(collect_list(struct(timestamp, event_page))).event_page) as route
                  FROM clickstream
                  LEFT JOIN (SELECT user_id, session_id, MIN(timestamp) as error_ts
                             FROM clickstream
                             WHERE event_type LIKE '%error%'
                             GROUP BY user_id, session_id) AS error_ts_table
                  ON clickstream.user_id = error_ts_table.user_id AND clickstream.session_id = error_ts_table.session_id
                  WHERE (event_type = 'page') AND (error_ts IS NULL OR timestamp < error_ts)
                  GROUP BY clickstream.user_id, clickstream.session_id)
            GROUP BY route
            ORDER BY count DESC
            '''
        )

sql_out.show(10)

+-------------------+-----+
|              route|count|
+-------------------+-----+
|               main| 8090|
|       main-archive| 1095|
|        main-rabota| 1039|
|      main-internet|  879|
|         main-bonus|  865|
|          main-news|  760|
|       main-tariffs|  669|
|        main-online|  584|
|         main-vklad|  514|
|main-archive-rabota|  167|
+-------------------+-----+
only showing top 10 rows



In [11]:
# write results
(sql_out.limit(30)
        .select(F.concat_ws(',', *sql_out.columns))
        .write.option('header', False)
        .text("sql_result", lineSep="\t"))

AnalysisException: path hdfs://localhost:9000/user/jovyan/sql_result already exists.

### Spark RDD Solution

In [12]:
# read csv into rdd
rdd = sc.textFile("hdfs:/data/clickstream.csv")

In [13]:
# split the values
rdd1 = rdd.map(lambda row: row.split('\t'))
# remove header
header = rdd1.first()
rdd2 = rdd1.filter(lambda row: row != header)
# rdd4 contains the timestamp of first error for user_id + session_id if any
rdd3 = rdd2.filter(lambda row: 'error' in row[2])
rdd4 = rdd3.map(lambda row: ((row[0], row[1]), int(row[-1]))).reduceByKey(lambda a, b: min(a, b))
rdd4.take(5)

[(('3491', '862'), 1695594151),
 (('455', '220'), 1695614507),
 (('2564', '10'), 1695620464),
 (('327', '955'), 1695637653),
 (('2118', '520'), 1695644046)]

In [14]:
# rdd5 is our original data without header, and mapped by (user_id, session_id)
# rdd6 join rdd5 with rdd4
rdd5 = rdd2.map(lambda row: ((row[0], row[1]), (row[2], row[3], int(row[4]))))
rdd6 = rdd5.leftOuterJoin(rdd4)

In [15]:
# rdd7 only keeps event which are 'page' from rdd6
rdd7 = rdd6.filter(lambda row: (row[1][0][0] == 'page'))
rdd7.take(5)

[(('2218', '958'), (('page', 'main', 1695586693), None)),
 (('3491', '862'), (('page', 'main', 1695593372), 1695594151)),
 (('3491', '862'), (('page', 'internet', 1695593474), 1695594151)),
 (('3491', '862'), (('page', 'news', 1695593625), 1695594151)),
 (('3491', '862'), (('page', 'rabota', 1695593640), 1695594151))]

In [16]:
# rdd8 filter rdd7 where there was no error in session and is mapped by (user_id, session_id) -> event_page
rdd8 = rdd7.filter(lambda row: (row[1][1] == None)).map(lambda row: (row[0], row[1][0][1]))
rdd8.take(5)

[(('2218', '958'), 'main'),
 (('1484', '744'), 'main'),
 (('1484', '744'), 'rabota'),
 (('1699', '297'), 'main'),
 (('1699', '297'), 'archive')]

In [18]:
# rdd9 filter rdd7 where there was error in session 
rdd9 = rdd7.filter(lambda row: (row[1][1] != None))

In [19]:
# rdd10 removes rows from rdd9 after the error and is mapped by (user_id, session_id) -> event_page
rdd10 = rdd9.filter(lambda row: (row[1][1] > row[1][0][2])).map(lambda row: (row[0], row[1][0][1]))
rdd10.take(5)

[(('3491', '862'), 'main'),
 (('3491', '862'), 'internet'),
 (('3491', '862'), 'news'),
 (('3491', '862'), 'rabota'),
 (('3491', '862'), 'archive')]

In [20]:
# rdd11 takes union of rdd8 and rdd10 and concatenate pages for each session and is mapped by (user_id, session_id) -> route
rdd11 = rdd10.union(rdd8).reduceByKey(lambda a, b: a + "-" + b)
rdd11.take(5)

[(('3491', '862'),
  'main-internet-news-rabota-archive-vklad-online-bonus-news-archive-internet-archive-tariffs'),
 (('455', '220'), 'main-archive'),
 (('2564', '10'),
  'main-tariffs-rabota-internet-rabota-archive-main-news-vklad-rabota-online-vklad-tariffs-news-online-vklad-internet'),
 (('327', '955'), 'main'),
 (('1932', '269'), 'main-main-internet-rabota-internet')]

In [21]:
# rdd12 calculates the count of routes in descending order

rdd12 = rdd11.map(lambda row: (row[1], 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda row: row[1], ascending=False)
rdd12.take(10)

[('main', 8090),
 ('main-archive', 1096),
 ('main-rabota', 1039),
 ('main-internet', 880),
 ('main-bonus', 865),
 ('main-news', 760),
 ('main-tariffs', 669),
 ('main-online', 583),
 ('main-vklad', 514),
 ('main-archive-rabota', 167)]

In [33]:
# write results
rdd12.map(lambda row : row[0] + "\t" + str(row[1])).take(30).saveAsTextFile('rdd_output')

AttributeError: 'list' object has no attribute 'saveAsTextFile'